# Titanic Survivorship
## Predicting who'd survive**

**URL**: https://www.kaggle.com/c/titanic/overview

In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import matplotlib.pylab as plt
import math

from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score

from sklearn import ensemble
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
%matplotlib inline
pd.options.display.max_columns = None

C:\Users\erikl\Anaconda2\envs\AppliedDataScience\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
test_raw = pd.read_csv("../input/test.csv")
train_raw = pd.read_csv("../input/train.csv")

In [3]:
train_raw.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Data Exploration
What does the data look like? How does each feature relate to survivorship?

In [4]:
# Let's check for missing values, we will have to address these missing values later 
null_cols = set(train_raw.columns[train_raw.isna().any()].tolist())
null_cols.update(test_raw.columns[test_raw.isna().any()].tolist())
print(null_cols)
train_raw.columns[train_raw.isna().any()]

{'Age', 'Embarked', 'Fare', 'Cabin'}


Index(['Age', 'Cabin', 'Embarked'], dtype='object')

In [5]:
# Let's first check the easier label columns 
print(train_raw['Survived'].groupby(train_raw['Pclass']).mean(), '\n')
print(train_raw['Survived'].groupby(train_raw['Sex']).mean(), '\n')
print(train_raw['Survived'].groupby(train_raw['Embarked']).mean())

# We can see that Pclass, Sex, and Embarking Port were very important
# for determining survival 

Pclass
1    0.629630
2    0.472826
3    0.242363
Name: Survived, dtype: float64 

Sex
female    0.742038
male      0.188908
Name: Survived, dtype: float64 

Embarked
C    0.553571
Q    0.389610
S    0.336957
Name: Survived, dtype: float64


In [6]:
# Now let's look at numeric columns, but first we must cut them

def get_age_range(data):
    bins = [0, 18, 35, 50, np.inf]
    names = ['0-18', '18-35', '35-50', '50+']
    return pd.cut(data['Age'], bins, labels=names)

train_raw['age_range'] = get_age_range(train_raw)
print(train_raw[['Survived','age_range']].groupby(['age_range']).mean())
# print(train_raw[['Survived','age_range','Pclass']].groupby(['age_range','Pclass']).mean())

del train_raw['age_range'] # let's clean that up 

# Looks like being young is an advantage to survive the titanic 

           Survived
age_range          
0-18       0.503597
18-35      0.382682
35-50      0.398693
50+        0.343750


In [7]:
# What about your family size?

def get_family_size(data):
    return data.apply(lambda row: 
                      "alone" if (row["SibSp"] + row["Parch"]) == 0 else "small" 
                      if (row["SibSp"] + row["Parch"]) <= 3 
                      else "large", axis=1 )

train_raw["family_size"] = get_family_size(train_raw)
        

print(train_raw[['Survived','family_size']].groupby(['family_size']).mean())
del train_raw['family_size']

# Small families had higher survival rates, followed by solo travelers, and finally large families 

             Survived
family_size          
alone        0.303538
large        0.161290
small        0.578767


In [8]:
# What about Fare paid?

train_raw['fare_range'] = pd.qcut(train_raw['Fare'], 6)
print(train_raw[['Survived','fare_range']].groupby(['fare_range']).mean())
del train_raw['fare_range']

# Of course the more you paid, the richer you were, the more likely you survived

                   Survived
fare_range                 
(-0.001, 7.775]    0.205128
(7.775, 8.662]     0.190789
(8.662, 14.454]    0.366906
(14.454, 26.0]     0.436242
(26.0, 52.369]     0.417808
(52.369, 512.329]  0.697987


In [9]:
# For cabin, let's separate the number and letter

def get_cabin_letter(data):
    return data.apply(lambda row: "Missing" if str(row['Cabin'])[0] == "n" else str(row['Cabin'])[0], axis=1)

def get_cabin_num(data):
    raw_num = data['Cabin'].apply(lambda x: str(x).split(' ')[-1][1:])
    raw_num.replace('an', np.NaN, inplace = True)
    return raw_num.apply(lambda x: int(x) if not pd.isnull(x) and x != '' else np.NaN)

train_raw['cabin_letter'] = get_cabin_letter(train_raw)
print(train_raw[['Survived','cabin_letter']].groupby(['cabin_letter']).mean().sort_values(by='Survived',ascending=False)
      , '\n')
del train_raw['cabin_letter']

# We see that D, E, B cabins had higher survival rates, and those with missing cabins had the lowest
# But what about cabin numbers?

train_raw['cabin_num'] = pd.qcut(get_cabin_num(train_raw), 5)
print(train_raw[['Survived','cabin_num']].groupby(['cabin_num']).mean().sort_values(by='Survived',ascending=False))
del train_raw['cabin_num']

# Looks like certain cabin numbers also had higher survival rates

              Survived
cabin_letter          
D             0.757576
E             0.750000
B             0.744681
F             0.615385
C             0.593220
G             0.500000
A             0.466667
Missing       0.299854
T             0.000000 

               Survived
cabin_num              
(1.999, 19.8]  0.725000
(33.0, 52.0]   0.700000
(19.8, 33.0]   0.682927
(85.2, 148.0]  0.650000
(52.0, 85.2]   0.589744


In [10]:
# Let's do something similar to ticket
def get_ticket_len(data):
    return data.apply(lambda row: len(row['Ticket']), axis=1)

def get_ticket_letter(data):
    return data.apply(lambda row: str(str(row['Ticket'])[0]), axis=1)

def map_ticket_letter(letter):
    if letter in ['3','2','1','S','P','C','A']:
        return letter
    elif letter in ['W','4','7','F','6','L','5','8','9']:
        return "uncommon" 
    else:
        return "unknown"

# Does the ticket length tell us anything?
train_raw['ticket_len'] = get_ticket_len(train_raw)
print(train_raw[['Survived','ticket_len']].groupby(['ticket_len']).mean(), '\n')
del train_raw['ticket_len']
#Looking at the results, a bit

#What about the first ticket letter?
train_raw['ticket_letter'] = get_ticket_letter(train_raw)
print(train_raw[['Survived','ticket_letter']].groupby(['ticket_letter']).mean().sort_values(by='Survived',ascending=False), '\n')
#There's definitely some info here, but we may have to cut down on which ticket letter matter, by looking at the counts
print(train_raw[['Survived','ticket_letter']].groupby(['ticket_letter']).count().sort_values(by='Survived',ascending=False), '\n')



del train_raw['ticket_letter']

            Survived
ticket_len          
3           0.000000
4           0.366337
5           0.618321
6           0.319809
7           0.296296
8           0.539474
9           0.192308
10          0.341463
11          0.250000
12          0.400000
13          0.400000
15          0.333333
16          0.272727
17          0.428571
18          0.000000 

               Survived
ticket_letter          
9              1.000000
P              0.646154
1              0.630137
F              0.571429
2              0.464481
C              0.340426
S              0.323077
L              0.250000
3              0.239203
4              0.200000
6              0.166667
W              0.153846
7              0.111111
A              0.068966
5              0.000000
8              0.000000 

               Survived
ticket_letter          
3                   301
2                   183
1                   146
P                    65
S                    65
C                    47
A              

In [11]:
# Let's examine the info we can extract from Name

def get_title(data):
    return data.apply(lambda row: row['Name'].split(",")[1].strip().split(" ")[0], axis=1)

def get_name_len(data):
    return data.apply(lambda row: len(row['Name']), axis=1)

# We can get the title from Name
train_raw['title'] = get_title(train_raw)
print(train_raw[['Survived','title']].groupby(['title']).mean().sort_values(by='Survived',ascending=False))
del train_raw['title']
# We can see certain nobility titles seem to have way better odds of survival

# What about how long the name is?
train_raw['name_len'] = pd.qcut(get_name_len(train_raw), 5)
print(train_raw[['Survived','name_len']].groupby(['name_len']).mean().sort_values(by='Survived',ascending=False))
del train_raw['name_len']
# We see that long names have higher survival rates, maybe people with longer names were also richer? 

           Survived
title              
the        1.000000
Mlle.      1.000000
Sir.       1.000000
Ms.        1.000000
Lady.      1.000000
Mme.       1.000000
Mrs.       0.792000
Miss.      0.697802
Master.    0.575000
Col.       0.500000
Major.     0.500000
Dr.        0.428571
Mr.        0.156673
Jonkheer.  0.000000
Rev.       0.000000
Don.       0.000000
Capt.      0.000000
                Survived
name_len                
(32.0, 82.0]    0.674556
(27.0, 32.0]    0.442424
(23.0, 27.0]    0.319797
(19.0, 23.0]    0.301282
(11.999, 19.0]  0.220588


## Data Preprocessing

In [12]:
# These functions transform our columns into the final form we need them in 
def trans_name(train, test):
    for data in [train, test]:
            data["title"] = get_title(data)
            data["name_len"] = get_name_len(data)
            del data["Name"]
    return train,test

def trans_age(train, test):
    for data in [train, test]:
        data["age_missing"] = data.apply(lambda row: 1 if row['Age'] != row['Age'] else 0, axis=1)
        newAges = train.groupby(['title', 'Pclass'])['Age']
        data['Age'] = newAges.transform(lambda x: x.fillna(x.mean()))
    return train,test
    
def trans_ticket(train, test):
    for data in [train, test]:
        data["ticket_len"] = get_ticket_len(data)
        data["ticket_letter"] = get_ticket_letter(data)
        data["ticket_letter"] = data.apply(lambda row: map_ticket_letter(row['ticket_letter']), axis=1)
        del data['Ticket']
    return train, test

def trans_family(train, test):
    for data in [train, test]:
        data["family_size"] = get_family_size(data)
        del data["SibSp"]
        del data["Parch"]
    return train,test

def trans_fare(train, test):
    mean = train['Fare'].mean()    
    train['Fare'].fillna(mean, inplace=True) 
    test['Fare'].fillna(mean, inplace=True) 
    return train,test

def trans_embarked(train, test):
    train['Embarked'] = train['Embarked'].fillna("S") 
    test['Embarked'] = test['Embarked'].fillna("S") 
    return train,test

def trans_cabin(train, test):
    for data in [train, test]:
        data["cabin_missing"] = data.apply(lambda row: 0 if row['Cabin'] == row['Cabin'] else 1, axis=1)
        data["cabin_letter"] = get_cabin_letter(data)
        data["cabin_num1"] = get_cabin_num(data)
        data['cabin_num'] = pd.qcut(train['cabin_num1'],5)
    
    train = pd.concat((train, pd.get_dummies(train['cabin_num'], prefix = 'cabin_num')), axis = 1)
    test = pd.concat((test, pd.get_dummies(test['cabin_num'], prefix = 'cabin_num')), axis = 1)
    del train['cabin_num']
    del test['cabin_num']
    del train['cabin_num1']
    del test['cabin_num1']
    del test['Cabin']
    del train['Cabin']
    return train, test

def get_dummies(train, test):
    cols_to_expand = ['Sex','Embarked', 'Pclass', 'title', 'cabin_letter','family_size', 'ticket_letter']
    for column in cols_to_expand:
        vals = set(train[column].unique())
        vals = vals.intersection(set(test[column].unique()))
        new_cols = [column + "_" + str(val) for val in vals]

        train = pd.concat((train, pd.get_dummies(train[column], prefix = column)[new_cols]), axis = 1)
        test = pd.concat((test, pd.get_dummies(test[column], prefix = column)[new_cols]), axis = 1)
    
        del train[column]
        del test[column]

    return train, test

In [ ]:
def scale_data(train, test, cols):
    cols = list(train.columns)
    cols.remove("PassengerId")
    cols.remove("Survived")
    all_cols = get_cols_by_type(test[cols])
    num_cols = all_cols['int64']
    num_cols.extend(all_cols['float64'])

    scaler = MinMaxScaler()
    X_train = train.copy()
    X_test = test.copy()
    X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
    X_test[num_cols] = scaler.transform(X_test[num_cols])
    return X_train, X_test


In [56]:
train = train_raw.copy()
test = test_raw.copy()

train, test = trans_name(train, test)
train, test = trans_age(train, test)  
train, test = trans_ticket(train, test)
train, test = trans_family(train, test)
train, test = trans_fare(train, test)
train, test = trans_embarked(train, test)
train, test = trans_cabin(train, test)
train, test = get_dummies(train, test)
train, test = scale_data(train, test, list(train.columns))

print("Columns: ", len(train.columns))
cols = list(train.columns[2:]) # features relevant to our ML model

# scaler = MinMaxScaler()
# train[cols] = scaler.fit_transform(train[cols])
# test[cols] = scaler.transform(test[cols])

print("Any Nulls: ", train.isnull().values.any(), test.isnull().values.any())
train.head()

Columns:  48
Any Nulls:  False False


,PassengerId,Survived,Age,Fare,name_len,age_missing,ticket_len,cabin_missing,"cabin_num_(1.999, 19.8]","cabin_num_(19.8, 33.0]","cabin_num_(33.0, 52.0]","cabin_num_(52.0, 85.2]","cabin_num_(85.2, 148.0]",Sex_female,Sex_male,Embarked_S,Embarked_C,Embarked_Q,Pclass_1,Pclass_2,Pclass_3,title_Col.,title_Rev.,title_Ms.,title_Miss.,title_Mr.,title_Master.,title_Dr.,title_Mrs.,cabin_letter_B,cabin_letter_D,cabin_letter_A,cabin_letter_F,cabin_letter_Missing,cabin_letter_E,cabin_letter_G,cabin_letter_C,family_size_small,family_size_alone,family_size_large,ticket_letter_S,ticket_letter_uncommon,ticket_letter_3,ticket_letter_A,ticket_letter_P,ticket_letter_2,ticket_letter_1,ticket_letter_C
0,1,0,22.0,7.2500,23,0,9,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0
1,2,1,38.0,71.2833,51,0,8,0,0,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0
2,3,1,26.0,7.9250,22,0,16,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0
3,4,1,35.0,53.1000,44,0,6,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0
4,5,0,35.0,8.0500,24,0,6,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0


### Feature Selection

In [ ]:
# We will use these functions to select the best features from our data
def select_cols(feature_cols, target, data, k):
    from sklearn.feature_selection import SelectKBest, f_classif

    selector = SelectKBest(f_classif, k=k)
    X_new = selector.fit_transform(data[feature_cols], data[target]) 

    selected_features = pd.DataFrame(selector.inverse_transform(X_new), index=data.index, columns=feature_cols)
    selected_columns = selected_features.columns[selected_features.var() != 0]
    return selected_columns
    
def find_best_cols(cols, target, data):
    import warnings
    warnings.filterwarnings("ignore")
    state = 1993  
    size = 0.30 
   
    X_train = data[:1000]
    X_valid = data[1000:]
    
    lr = ensemble.GradientBoostingRegressor() #Base Model
    lr.fit(X_train[cols], X_train[target].values.ravel())
    print ("Base Score: ", lr.score(X_train[cols], X_train[target].values.ravel())) # 0.9672992360887501
    best_score = 0
    best_cols = cols
    for k in range(len(cols)//4, len(cols)):
        lr = ensemble.GradientBoostingRegressor()
        curr_cols = select_cols(cols, target, X_train, k)
        lr.fit(X_train[curr_cols], X_train[target].values.ravel())
        os_score = lr.score(X_valid[curr_cols], X_valid[target].values.ravel())
        if os_score > best_score:
            is_score = lr.score(X_train[curr_cols], X_train[target].values.ravel())
            print ("K= ", k, ", IS score: ", is_score, ", OS score: ", os_score) # 0.840628507295174
            best_score = os_score
            best_cols = curr_cols
            
    return best_cols

best_cols = find_best_cols(cols, "SalePrice", train)
best_cols

In [ ]:
print(len(best_cols))
print(len(cols))
cols = best_cols

In [57]:
# We'll use this function to save our results to CSV
def save_results(model, data):
    pred_test = model.predict(data)

    #PassengerId,Survived
    test_res = test[["PassengerId"]].copy()
    test_res["Survived"] = pred_test
    test_res.to_csv("my_predictions.csv", index=False)
    return test_res

## Modelling

In [74]:
# We use this function to tune our model
def get_tuned_model(estimator, param_grid, scoring, X_train, Y_train):
    from sklearn.model_selection import GridSearchCV

    grid = GridSearchCV(estimator = estimator, 
                       param_grid = param_grid,
                       scoring = scoring,
                       cv=3,
                       n_jobs= -1
                      )

    tuned = grid.fit(X_train, Y_train)

    print ("Best score: ", tuned.best_score_) # 0.840628507295174
    print ("Best params: ", tuned.best_params_)
    print ("IS Score: ", tuned.score(X_train, Y_train)) # 0.8698092031425365
    
    return tuned

### Logistic Regression 

In [64]:
logit = LogisticRegression()

param_grid = {'penalty': ['l1','l2'], 
              'C': [0.001,0.01,0.1,1,10,100],
              'max_iter': [100,200,300,500]
             }

log_tuned = get_tuned_model(logit, param_grid, "accuracy", train[cols], train[['Survived']].values.ravel())

Best score:  0.8316427091833531
Best params:  {'C': 0.1, 'max_iter': 100, 'penalty': 'l2'}
IS Score:  0.8439955106621774


C:\Users\erikl\Anaconda2\envs\AppliedDataScience\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [62]:
save_results(log_tuned, test[cols])

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


### Random Forest Classifier

In [65]:
param_grid = { 
    "criterion" : ["gini", "entropy"], 
    "min_samples_leaf" : [1, 5, 10], 
#     "min_samples_split" : [2, 4, 10, 12, 16],
#     "n_estimators": n_estimators,
    'max_leaf_nodes': range(2,20)
}

forest = RandomForestClassifier()
ft_tuned = get_tuned_model(forest, param_grid, "accuracy", train[cols], train[['Survived']].values.ravel())
# best 0.840628507295174

Best score:  0.8293955181721173
Best params:  {'criterion': 'gini', 'max_leaf_nodes': 14, 'min_samples_leaf': 1}
IS Score:  0.856341189674523


In [ ]:
save_results(ft_tuned, test[cols])

### Gradient Boost 

In [70]:
param_grid = { 
    "learning_rate":  [0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1],
    "n_estimators": [8, 16, 32, 64, 100, 200, 400, 500],
#     'subsample':[0.5, 0.6, 0.7,0.75,0.8,0.85,0.9,0.95,1],
    'max_features': list(range(len(cols)//4,len(cols)//2)),
}

gbc = ensemble.GradientBoostingClassifier()
gbc_tuned = get_tuned_model(gbc, param_grid, "accuracy", train[cols], train[['Survived']].values.ravel())

Best score:  0.857447743393384
Best params:  {'learning_rate': 0.1, 'max_features': 15, 'n_estimators': 500}
IS Score:  0.9685746352413019


In [71]:
save_results(gbc_tuned, test[cols])

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
